In [ ]:
# import library
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Pra Pengolahan Data

In [ ]:
# Konfigurasi augmentasi dan praproses gambar untuk data pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Normalisasi piksel gambar ke rentang 0-1
    shear_range=0.2,        # Efek pemotongan gambar
    zoom_range=0.2,         # Efek zoom pada gambar
    horizontal_flip=True    # Flipping horizontal gambar
)

# Memuat data pelatihan langsung dari direktori
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Datasets/animal_dataset/training_set',  # Lokasi direktori data pelatihan
    target_size=(64, 64),     # Ukuran target gambar setelah dimuat
    batch_size=32,            # Jumlah gambar dalam setiap batch
    class_mode='binary'       # Mode kelas untuk masalah klasifikasi biner
)


Found 8000 images belonging to 2 classes.


In [ ]:
# Konfigurasi praproses gambar untuk data pengujian
test_datagen = ImageDataGenerator(
    rescale=1./255  # Normalisasi piksel gambar ke rentang 0-1
)

# Memuat data pengujian langsung dari direktori
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Datasets/animal_dataset/test_set',  # Lokasi direktori data pengujian
    target_size=(64, 64),  # Ukuran target gambar setelah dimuat
    batch_size=32,  # Jumlah gambar dalam setiap batch
    class_mode='binary'  # Mode kelas untuk masalah klasifikasi biner
)


Found 2000 images belonging to 2 classes.


### Pembuatan Model CNN

In [ ]:
# Membuat objek model sequential
cnn = tf.keras.models.Sequential()

In [ ]:
# Layer Konvolusi pertama
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [ ]:
# MaxPooling untuk mereduksi dimensi spasial gambar
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# Layer Konvolusi kedua
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# Meratakan output untuk dimasukkan ke dalam layer Dense/Fully Connected
cnn.add(tf.keras.layers.Flatten())

In [ ]:
# Dense layer dengan 128 unit dan aktivasi ReLU
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
# Dense layer dengan 1 unit dan aktivasi sigmoid untuk klasifikasi biner
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
# Kompilasi model dengan optimizer 'adam', fungsi kerugian 'binary_crossentropy', dan metrik akurasi
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### Fit CNN

In [ ]:
# Pelatihan Model CNN dengan data pelatihan dan validasi
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25
250/250 [==============================] - 436s 2s/step - loss: 0.5959 - accuracy: 0.6787 - val_loss: 0.5480 - val_accuracy: 0.7270
Epoch 2/25
250/250 [==============================] - 38s 151ms/step - loss: 0.5681 - accuracy: 0.7035 - val_loss: 0.5664 - val_accuracy: 0.7045
Epoch 3/25
250/250 [==============================] - 38s 150ms/step - loss: 0.5244 - accuracy: 0.7376 - val_loss: 0.5133 - val_accuracy: 0.7490
Epoch 4/25
250/250 [==============================] - 37s 149ms/step - loss: 0.5116 - accuracy: 0.7473 - val_loss: 0.5508 - val_accuracy: 0.7405
Epoch 5/25
250/250 [==============================] - 38s 150ms/step - loss: 0.4893 - accuracy: 0.7626 - val_loss: 0.4815 - val_accuracy: 0.7725
Epoch 6/25
250/250 [==============================] - 37s 148ms/step - loss: 0.4776 - accuracy: 0.7711 - val_loss: 0.6236 - val_accuracy: 0.7000
Epoch 7/25
250/250 [==============================] - 38s 151ms/step - loss: 0.4671 - accuracy: 0.7753 - val_loss: 0.4814 - val_accu

### Prediksi dengan 1 Citra

In [ ]:
# Import library yang dibutuhkan untuk prediksi
import numpy as np
from keras.preprocessing import image

# Memuat dan memproses gambar baru untuk prediksi
test_image = image.load_img('/content/drive/MyDrive/Datasets/animal_dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Melakukan prediksi menggunakan model
result = cnn.predict(test_image)

# Mendapatkan kelas indeks dari data pelatihan
classes = training_set.class_indices

# Menentukan hasil prediksi ('dog' atau 'cat')
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)

1/1 [==============================] - 0s 17ms/step
cat
